In [ ]:
%cp /content/drive/My\ Drive/save_weight/*prednet_kitti_weights_train4*  /content/
%cp /content/drive/My\ Drive/save_weight/*weights_train_delete_training_3_nt*  /content/
%cp /content/drive/My\ Drive/save_weight/*INITIAL_weights_train*  /content/
%cp /content/drive/My\ Drive/save_weight/*weights_train_SPECIAL6_2*  /content/
%cp /content/drive/My\ Drive/save_weight/*test_entire*  /content/
%cp /content/drive/My\ Drive/save_weight/*prednet_kitti_model*  /content/
%cp /content/drive/My\ Drive/save_weight/delete_training_model_3_nt.json  /content/
%cp /content/drive/My\ Drive/save_weight/INITIAL_model.json*  /content/
%cp /content/drive/My\ Drive/save_weight/training_model_SPECIAL6.json*  /content/
%cp /content/drive/My\ Drive/save_weight/*best_validation*  /content/


cp: cannot stat '/content/drive/My Drive/save_weight/*weights_train_SPECIAL6_2*': No such file or directory


In [ ]:
!python3 utils.py
!python3 prednet.py 

W0806 15:05:56.251872 140424643852160 deprecation_wrapper.py:119] From utils.py:9: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

Using TensorFlow backend.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.4M  100 80.4M    0     0  17.0M      0  0:00:04  0:00:04 --:--:-- 17.8M
Using TensorFlow backend.


In [ ]:
from utils import *
   
def perceptual_distance(y_true, y_pred):
    # Muliply by 225 because normalization divides by 225.
    # Taking the last output of the LSTM

    y_pred  =denormalize (y_pred)
    y_true  =denormalize (y_true) 
    #print("y_true_1",K.int_shape(y_pred1))
    #print("y_true_1",K.int_shape(y_true1))
    
    rmean = (y_true[:, :, :, 0] + y_pred[:, :, :, 0]) / 2
    r = y_true[:, :, :, 0] - y_pred[:, :, :, 0]
    g = y_true[:, :, :, 1] - y_pred[:, :, :, 1]
    b = y_true[:, :, :, 2] - y_pred[:, :, :, 2]

    return K.mean(K.sqrt((((512+rmean)*r*r)/256) + 4*g*g + (((767-rmean)*b*b)/256)))

from keras.models import load_model
new_model = load_model('/content/best_validation.hdf5',custom_objects={'perceptual_distance':perceptual_distance,'loss':loss}) 



W0806 15:08:55.680596 139947945498496 deprecation_wrapper.py:119] From /content/utils.py:9: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

Using TensorFlow backend.
W0806 15:09:04.326379 139947945498496 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 15:09:04.334940 139947945498496 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 15:09:04.343795 139947945498496 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0806 15:09:04.742151 139947945498496 deprecation_wrapper.py:119] From /u

alpha 1
(?, 96, 96, 3)
SHAPE_OF_GDL_LOSS (?, 96, 96)
y_true (?, ?, ?, ?)
y_pred (?, 96, 96, 3)
SHAPE_OF_LOSS (None, 96, 96)


W0806 15:09:11.183794 139947945498496 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
import hickle as hkl
from keras.models import Model, model_from_json
import os
from prednet import *

test_file    = os.path.join('/content/test_entire2_X_test.hkl')
test_sources = os.path.join('/content/test_entire2_sources_test.hkl')


#nt = 4 for training but for training no of images = 5
weights_file = os.path.join('/content/prednet_kitti_weights_train4.hdf5')
json_file    = os.path.join('/content/prednet_kitti_model.json')


#nt = 3 for training but for training no of images = 6,epoch = 50 ,samples_per_epoch = 2433*4, layer_dimension =64,128,256

weights_file2 = os.path.join('/content/weights_train_delete_training_3_nt.hdf5')
json_file2 = os.path.join('/content/delete_training_model_3_nt.json')


weights_file3 = os.path.join('/content/INITIAL_weights_train.hdf5')
json_file3 = os.path.join('/content/INITIAL_model.json')




weights_file4 = os.path.join('/content/weights_train_SPECIAL6_2.hdf5')
json_file4    = os.path.join('/content/training_model_SPECIAL6.json')



sources = hkl.load(test_sources)
X_array      = hkl.load(test_file)


nt = 4
# Load trained model
f = open(json_file, 'r')
json_string = f.read()
f.close()
train_model = model_from_json(json_string, custom_objects = {'PredNet': PredNet})
train_model.load_weights(weights_file)
# Create testing model (to output predictions)
layer_config = train_model.layers[1].get_config()
layer_config['output_mode'] = 'prediction'
data_format = layer_config['data_format'] if 'data_format' in layer_config else layer_config['dim_ordering']
test_prednet = PredNet(weights=train_model.layers[1].get_weights(), **layer_config)
input_shape = list(train_model.layers[0].batch_input_shape[1:])
input_shape[0] = nt
inputs = Input(shape=tuple(input_shape))
predictions = test_prednet(inputs)
test_model = Model(inputs=inputs, outputs=predictions)




nt = 3
# Load trained model
f = open(json_file2, 'r')
json_string2 = f.read()
f.close()
train_model2 = model_from_json(json_string2, custom_objects = {'PredNet': PredNet})
train_model2.load_weights(weights_file2)

# Create testing model (to output predictions)
layer_config2 = train_model2.layers[1].get_config()
layer_config2['output_mode'] = 'prediction'
data_format2 = layer_config2['data_format'] if 'data_format' in layer_config2 else layer_config2['dim_ordering']
test_prednet2 = PredNet(weights=train_model2.layers[1].get_weights(), **layer_config2)
input_shape2 = list(train_model2.layers[0].batch_input_shape[1:])
input_shape2[0] = nt
inputs2 = Input(shape=tuple(input_shape2))
predictions2 = test_prednet2(inputs2)
test_model2 = Model(inputs=inputs2, outputs=predictions2)

nt = 3

# Load trained model
f = open(json_file3, 'r')
json_string3 = f.read()
f.close()
train_model3 = model_from_json(json_string3, custom_objects = {'PredNet': PredNet})
train_model3.load_weights(weights_file3)


# Create testing model (to output predictions)
layer_config3 = train_model3.layers[1].get_config()
layer_config3['output_mode'] = 'prediction'
data_format3 = layer_config3['data_format'] if 'data_format' in layer_config3 else layer_config3['dim_ordering']
test_prednet3 = PredNet(weights=train_model3.layers[1].get_weights(), **layer_config3)
input_shape3 = list(train_model3.layers[0].batch_input_shape[1:])
input_shape3[0] = nt
inputs3 = Input(shape=tuple(input_shape3))
predictions3 = test_prednet3(inputs3)
test_model3 = Model(inputs=inputs3, outputs=predictions3)




nt = 3

f = open(json_file4, 'r')
json_string4 = f.read()
f.close()
train_model4 = model_from_json(json_string4, custom_objects = {'PredNet': PredNet})
train_model4.load_weights(weights_file4)


# Create testing model (to output predictions)
layer_config4 = train_model4.layers[1].get_config()
layer_config4['output_mode'] = 'prediction'
data_format4 = layer_config4['data_format'] if 'data_format' in layer_config4 else layer_config4['dim_ordering']
test_prednet4 = PredNet(weights=train_model4.layers[1].get_weights(), **layer_config4)
input_shape4 = list(train_model4.layers[0].batch_input_shape[1:])
input_shape4[0] = nt
inputs4 = Input(shape=tuple(input_shape4))
predictions4 = test_prednet4(inputs4)
test_model4 = Model(inputs=inputs4, outputs=predictions4)



In [ ]:
def list_start(sources):
  
  start_list =[]
  for i in range(len(sources)-1):
    if(sources[i]!=sources[i+1]):
      start = i+1
      start_list.append(start)
  start_list.append(len(sources)-1)
  return start_list  

In [ ]:
def find_closest_img_to_op(input_imgs,output_images):
   percep0 = perceptual_distance(input_imgs[:,0,:,:,:],output_images)
   percep1 = perceptual_distance(input_imgs[:,1,:,:,:],output_images)
   percep2 = perceptual_distance(input_imgs[:,2,:,:,:],output_images)
   percep3 = perceptual_distance(input_imgs[:,3,:,:,:],output_images)
   percep4 = perceptual_distance(input_imgs[:,4,:,:,:],output_images)

   return np.argmin(np.array((percep0,percep1,percep2,percep3,percep4)))

def my_predict(img_list):
    """A generator that returns 5 images plus a result image"""
    "" " Returns a input_images of shape(batch_size,no_of_time_seq,height,width,depth) """

    cat_dirs = img_list
    
    counter = 0
    j=0
    percep_list =[]
   
    possible_starts  =  list_start(sources)
    
    
    while True:
        input_images = np.zeros((1,5 ,width, height,3 ),dtype='float32')
        aug_input_image =np.zeros((1,5 ,width, height,3 ),dtype='float32')
        output_images   = np.zeros((1,width, height, 3),dtype='float32')
        input_images2   = np.zeros((1,width, height, 3),dtype='float32')
        
        X_all = np.zeros((1,3 ,96, 96,3 ),dtype='float32')
        X_all2 = np.zeros((1,3 ,96, 96,3 ),dtype='float32')  
        
          
        if (counter+batch_size >= len(cat_dirs)):
            counter = 0
            j       = 0
       
        for i in range(batch_size):
            end  =  possible_starts[j]
            start =  possible_starts[j]-3
            input_imgs = glob.glob(cat_dirs[counter + i] + "/cat_[0-5]*")
            input_images = np.expand_dims(np.array([normalized(np.array(Image.open(img))) for img in sorted(input_imgs)]),axis=0)
            act_out =np.array(Image.open(cat_dirs[counter + i] + "/cat_result.jpg"))
            act_out =normalized(act_out)
            output_images    = np.expand_dims(act_out, axis=0)
            
            index = find_closest_img_to_op(input_images,output_images)
            input_images2    = input_images[:,index,:,:,:]

            
            #Model1
            y1 = new_model.predict([input_images,input_images2])
            percep1 = perceptual_distance(output_images,y1)
            
            #Model2
            
            end  =  possible_starts[j]
            start =  possible_starts[j]-3
            X_all2 = np.expand_dims(normalized(X_array[start:end]),axis=0)    
            
            
            X_hat  =test_model3.predict(X_all2)
            percep2  = perceptual_distance(X_hat[:,-1,:,:,:], X_all2[:,-1,:,:,:])
           
            #Model3
            X_hat2   =test_model2.predict(X_all2)
            percep3  = perceptual_distance(X_hat2[:,-1,:,:,:], X_all2[:,-1,:,:,:])
            
            #Model5
            
            X_hat4   =test_model4.predict(X_all2)
            percep5  = perceptual_distance(X_hat4[:,-1,:,:,:], X_all2[:,-1,:,:,:])
           
            
            
            #Model4
            start =  possible_starts[j]-4
            X_all = np.expand_dims(normalized(X_array[start:end]),axis=0)
            X_hat3   =test_model.predict(X_all)
            percep4  = perceptual_distance(X_hat3[:,-1,:,:,:], X_all[:,-1,:,:,:])
            
           
            
            start =  possible_starts[j]-3
            
           
            per_list = [percep1,percep2,percep3,percep4,percep5]
            percep_list.append(min(per_list))
            print(sources[start],min(per_list),percep1,percep2,percep3,percep4,percep5)
            
            
            
            j = (j+1)
                 
        counter += batch_size
        yield percep_list

In [ ]:
!python3 utils.py

W0806 15:09:55.200101 140063178397568 deprecation_wrapper.py:119] From utils.py:9: The name tf.set_random_seed is deprecated. Please use tf.compat.v1.set_random_seed instead.

Using TensorFlow backend.


In [ ]:
from utils import *

co =0
for i in my_predict(f7(sources)):
  co +=batch_size
  
  if(co >= len(val_list)):
    break
  
print (sum(i)/len(i))